<a href="https://colab.research.google.com/github/yuki0703-sudo/Research/blob/main/MarkerTracking_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [ ]:
from google.colab import drive
import os
import glob
import cv2
import numpy as np
from IPython import display
from matplotlib import pyplot as plt
import time
from numpy import dtype,uint8
from PIL import Image 
from PIL import ImageDraw
import sys
import numpy as np
import csv
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
src_img_dir="/content/gdrive/My Drive/YAMAKAWA_LAB/30_0722/1_files/000001.tif"
#src_files=glob.glob("{}/*.tif".format(src_img_dir))
#src_files.sort()
img=cv2.imread(src_img_dir)
print(img.shape)
img=np.array(img,dtype=uint8)
print(img.shape)

(508, 632, 3)
(508, 632, 3)


#Marker Tracking of Nail positions

In [ ]:
#ffmpegを使って動画を画像に変換する
s=0 #周波数計算用，sum
fps=540 #取得画像のfps
dat=[] #[x_average,y_average,x_velocity,y_velocity]
#マーカートラッキング評価用
accuracy=0
error_list=[]

src_img_dir="/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/"

#src_img_dirがなかったら、srcフォルダを作成する
#if not os.path.exists(src_img_dir):
#  os.makedirs(src_img_dir)

#抽出された画像をsrcディレクトリに保存#fpsを指定,今回は536fps
#!ffmpeg -i "/content/gdrive/My Drive/YAMAKAWA_LAB/motion_tracking/sample.mp4" -vf fps=530 "$src_img_dir/%04d.jpg" #-vf video to frame


#フォルダに存在するファイルを取得する、0001.jpg~0005.jpg→{}/*.jpg
src_files=glob.glob("{}/*.tif".format(src_img_dir))
src_files.sort()

#print(len(src_files), "image files")

#画像の保存ディレクトリ
dst_img_dir='/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3'
#out_img='/content/gdrive/My Drive/YAMAKAWA_LAB/motion_tracking/wavehand/output'

#出力ファイル名
filename='/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3.csv'

#data=np.arange(9).reshape(3,3)
#np.savetxt("/content/gdrive/My Drive/YAMAKAWA_LAB/motion_tracking/data.txt",data,fmt='%d')
#data=data.flatten()

#ファイルを作成
with open(filename,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["x_ave","y_ave","x1","y1","x2","y2","x3","y3","x4","y4"])

if not os.path.exists(dst_img_dir):
  os.makedirs(dst_img_dir)
#重心位置
position_x=[]
position_y=[]
#plt.imshow(img)




# encoder(for mp4)
#fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
#video = cv2.VideoWriter('output.mp4',fourcc, 30.0, (508,632))

for i,file in enumerate(src_files): #index,components=enumerate(file)
  print(file)
  t1=time.time()#周波数計算用，開始時間

  # 画像を読み込む。
  img=cv2.imread(file)
  img=np.array(img,dtype=uint8)

  # グレースケールに変換する。
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # 2値化する
  ret, bin_img = cv2.threshold(gray, 33, 255, cv2.THRESH_BINARY)

  # 輪郭を抽出する。
  contours, hierarchy = cv2.findContours(np.array(bin_img,dtype=uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # 小さい輪郭は誤検出として削除する
  contours = list(filter(lambda x: cv2.contourArea(x) >=2.5, contours))

  # 輪郭を描画する。
  #cv2.drawContours(img, contours, -1, color=(0, 0, 255), thickness=2)

  #面積、円形度、等価直径を求める。
  Areas = []
  Circularities = []
  #Eq_diameters = []

  x_c=0 #重心位置_x
  y_c=0 #重心位置_y
  e_c=0 #輪郭検出用変数

  

  #輪郭検出，マーカー位置取得，ループ構造

  #精度評価用
  count=0
  x_joint=[]
  y_joint=[]

  for c in contours:

    #面積(px*px)
    area = cv2.contourArea(c)
    #Areas.append(area)

    #円形度
    #arcLength(curve,ret) #輪郭の長さを返す，curve:輪郭情報，ret:平曲線かどうか
    arc = cv2.arcLength(c, True)
    circularity = 4 * np.pi * area / (arc * arc)
    #重心位置を求める
    if circularity > 0.05 and circularity<1.5: 

      # 輪郭を描画する。
      #cv2.drawContours(img, contours, 1, color=(0, 0, 255), thickness=2)

      #マーカー位置計算
      mu=cv2.moments(c)
      x=int(mu["m10"]/mu["m00"])
      y=int(mu["m01"]/mu["m00"])
      x_joint.append(x)
      y_joint.append(y)
      #精度評価用
      count+=1
      #cv2.circle(img,(x,y),10,(0,0,255),-1)
      #dst_file="{dir}/{index:04d}.jpg".format(dir=dst_img_dir,index=i)
      #img = np.array(img, np.uint8)
      #print(type(img))
      #img.reshape(508,632,3)
      #cv2.imwrite(dst_file,img)
      #画像に輪郭を書き込む
      #cv2.circle(img,(x,y),10,(255,0,0),-1)

      x_c+=x #平均位置を求める→合計
      y_c+=y #平均位置を求める→合計
      e_c+=1 #検出された輪郭の個数

      #x_l.append(x) #リストに重心座標を追加
      #y_l.append(y) #リストに重心座標を追加
  #ループ終了
  #print(c)

  x_c=int(x_c/e_c) #平均位置_x
  y_c=int(y_c/e_c)  #平均位置_y

  #重心座標を追加
  position_x.append(x_c)
  position_y.append(y_c)

  #if i!=0:
    #速度情報
   # v_x=(position_x[i]-position_x[i-1])#*fps
   # v_y=(position_y[i]-position_y[i-1])#*fps
   # dat.append([x_c,y_c,])
    

    #マーカートラッキング動画作成用
    #画像に重心位置を書き込む
  #cv2.circle(img,(x_c,y_c),10,(0,0,255),-1)
  #dst_file="{dir}/{index:04d}.jpg".format(dir=dst_img_dir,index=i)
  #img = np.array(img, np.uint8)
  #print(type(img))
  #img.reshape(img,(508,632,3))
  #cv2.imwrite(dst_file,img)
  
  #ファイルに重心座標を書き込む  
  with open(filename,"a",newline='') as f:
      writer = csv.writer(f)
      writer.writerow([x_c,y_c,x_joint[0],y_joint[0],x_joint[1],y_joint[1],x_joint[2],y_joint[2],x_joint[3],y_joint[3]])

  #マーカートラッキングの精度評価用
  
  if count!=4:
    accuracy+=1
    #print(i,len(contours))
  
  t2=time.time() #周波数計算用，終了時間
  s=s+(t2-t1) #sum

  #画像処理後の画像を保存
  #position_x.append(x_l)
  #position_y.append(y_l)

#print(position_x)
#video.release()
print("frequency from reading to writing =",1/(s/len(src_files))) #周波数 #円形度あり：68.25Hz,円形度なし:81.2Hz
print(accuracy,len(src_files))

/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000001.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000002.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000003.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000004.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000005.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000006.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000007.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000008.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000009.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000010.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000011.tif
/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3_files/000012.tif
/content/gdrive/

IndexError: ignored

In [ ]:
#処理後の画像を動画に変換する
import glob

dst_img_dir='/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3'
src_img_dir=dst_img_dir

#フォルダに存在するファイルを取得する、0001.jpg~0005.jpg→{}/*.jpg
src_files=glob.glob("{}/*.jpg".format(src_img_dir))
src_files.sort()

#処理後の画像から動画を作る
!ffmpeg -framerate 30 -i "$src_img_dir/%04d.jpg" "/content/gdrive/My Drive/YAMAKAWA_LAB/MarkerTracking_1007/1007/3.mp4"

In [ ]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
y1=[]
y2=[]
y3=[]
y4=[]
y5=[]
y6=[]

In [ ]:
for i in range(len(position_x)):
  x1.append(position_x[i][0])
  x2.append(position_x[i][1])
  x3.append(position_x[i][2])
  x4.append(position_x[i][3])
  x5.append(position_x[i][4])
  x6.append(position_x[i][5])
  y1.append(position_y[i][0])
  y2.append(position_y[i][1])
  y3.append(position_y[i][2])
  y4.append(position_y[i][3])
  y5.append(position_y[i][4])
  y6.append(position_y[i][5])

#print(x1)
#print(y1)
plt.scatter(x4,y4)
